In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('fake_news.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data.shape

(20800, 5)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
data.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [6]:
data = data.drop(['id'], axis=1)

In [7]:
data = data.fillna('')

In [8]:





data['content'] = data['author']+' '+data['title']+' '+data['text']

In [9]:
data = data.drop(['author', 'title', 'text'], axis=1)

In [11]:
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [10]:
data['content'] = data['content'].str.replace('[^\w\s]','')

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [14]:

!pip install textblob

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))
data['content'].head()

,content
0,darrell lucus house dem aide: didn’t even see ...
1,"daniel j. flynn flynn: hillary clinton, big wo..."
2,consortiumnews.com truth might get fired truth...
3,jessica purkiss 15 civilian killed single u ai...
4,howard portnoy iranian woman jailed fictional ...


In [17]:
X = data[['content']]
y = data['label']

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(16640, 1)
(4160, 1)
(16640,)
(4160,)


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(X_train['content'])
X_train_tfidf = tfidf_vect.transform(X_train['content'])
X_test_tfidf = tfidf_vect.transform(X_test['content'])

In [23]:
from sklearn import metrics

In [24]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf = PassiveAggressiveClassifier()
pclf.fit(X_train_tfidf,y_train)
predictions = pclf.predict(X_test_tfidf)
print(metrics.classification_report(y_test , predictions))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      2132
           1       0.96      0.97      0.97      2028

    accuracy                           0.97      4160
   macro avg       0.97      0.97      0.97      4160
weighted avg       0.97      0.97      0.97      4160



In [25]:
print(metrics.confusion_matrix(y_test , predictions))

[[2057   75]
 [  64 1964]]


In [30]:
import pickle
pickle.dump(pclf , open("fakenews1.pkl", "wb"))

In [32]:
loaded_model = pickle.load(open ("fakenews1.pkl" , "rb"))
result = loaded_model.score(X_test_tfidf , y_test)
print(result)

0.9665865384615384
